In [ ]:
# default_exp HeteroData_synthesize

In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

<IPython.core.display.Javascript object>

In [ ]:
# export
import numpy as np
from datetime import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from typing import Tuple, Optional, Dict
import pytorch_lightning as pl
import torch
from torch_geometric.utils import to_dense_batch
from torch_geometric.data import HeteroData
import torch_geometric.nn as pygnn
from torch_geometric.loader import DataLoader
from pathlib import Path


<IPython.core.display.Javascript object>

In [ ]:
#export
def synthesize_mixing_vector(length, random_state=46, **kwargs):
    """

    Args:
        length (int) : length = number_of_nodes*number of features
        random_state (int): seed for random shuffling

    Returns:
        np.ndarray mixing vector
    """
    indices = np.arange(length)
    return shuffle(indices, random_state=random_state)


def generate_positive_negative_data(length, total_samples=10000, **kwargs):
    """

    Args:
        length (int) : length = number_of_nodes*number of features
        total_samples (int): total number of instances
        **kwargs:

    Returns:
        Tuple[np.ndarray,np.ndarray] for positive and negative data
    """
    mixing_vec_pos = synthesize_mixing_vector(length, **kwargs)
    A_pos = np.random.randn(length, length)
    cov_pos = np.matmul(A_pos.T, A_pos)
    mean_pos = np.amax(np.abs(cov_pos.flatten())) * (1 + np.random.randn(length) * 0.25)
    raw_samples_pos = np.random.multivariate_normal(mean_pos, cov_pos, size=total_samples)
    samples_pos = raw_samples_pos[:, mixing_vec_pos]

    mixing_vec_neg = synthesize_mixing_vector(length, **kwargs)
    A_neg = np.random.randn(length, length)
    cov_neg = np.matmul(A_neg.T, A_neg)
    mean_neg = np.amax(np.abs(cov_neg.flatten())) * (-1 + np.random.randn(length) * 0.25)
    raw_samples_neg = np.random.multivariate_normal(mean_neg, cov_neg, size=total_samples)
    samples_neg = raw_samples_neg[:, mixing_vec_neg]
    return samples_pos, samples_neg


# noinspection PyPep8Naming
def get_data_list(number_of_nodes=15, number_of_features=5, total_samples=10000, **kwargs):
    X_p, X_n = generate_positive_negative_data(number_of_nodes * number_of_features, total_samples)
    Y = [0] * X_n.shape[0] + [1] * X_p.shape[0]
    X_p = np.reshape(X_p, (total_samples, number_of_nodes, number_of_features))
    X_n = np.reshape(X_n, (total_samples, number_of_nodes, number_of_features))
    X = np.concatenate([X_n, X_p], axis=0)
    edge_index_dict = {"type1": [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
                                 [0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4]],
                       "type2": [[0, 0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14],
                                 [1, 5, 7, 7, 7, 9, 6, 7, 9, 14, 11, 6, 7, 7, 13]]}
    data_list = [HeteroData(
        {"y": torch.tensor(y, dtype=torch.long),
         "node_a": {"x": torch.tensor(x, dtype=torch.float32)},
         ("node_a", "love", "node_a"): {"edge_index": torch.tensor(edge_index_dict["type1"], dtype=torch.long)},
         ("node_a", "hate", "node_a"): {"edge_index": torch.tensor(edge_index_dict["type2"], dtype=torch.long)}
         })
        for x, y in zip(X, Y)]
    return data_list


class SyntheticDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=100, **kwargs):
        super().__init__()
        self.batch_size = batch_size

        full_data_list = get_data_list(**kwargs)
        self.train_data_list, self.test_data_list = train_test_split(full_data_list, test_size=0.2, shuffle=True)
        self.static_edge_index_dict = full_data_list[0].edge_index_dict

    def train_dataloader(self):
        return DataLoader(self.train_data_list, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_data_list, batch_size=self.batch_size)

    def get_single_sample(self):
        return self.train_data_list[0]


class HeteroGNNArch(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers, number_of_labels=2):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = pygnn.HeteroConv({
                ("node_a", "love", "node_a"): pygnn.GCNConv(-1, hidden_channels),
                ("node_a", "hate", "node_a"): pygnn.SAGEConv((-1, -1), hidden_channels),
            }, aggr='sum')
            self.convs.append(conv)

        self.lin = pygnn.Linear(-1, hidden_channels)

        self.dense = torch.nn.LazyLinear(number_of_labels)
        self.softmax = torch.nn.Softmax(dim=0)

    def forward(self, x_dict, edge_index_dict, batch_dict: Optional[Dict] = None):
        # todo: make sure x_dict.keys() is orderd like batch_vector
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
        x_dict = {key: self.lin(xx) for key, xx in x_dict.items()}
        x_dict_batched = {key: torch.flatten(to_dense_batch(xx, b_vec)[0], start_dim=1) for (key, xx), b_vec in
                          zip(x_dict.items(), batch_dict.values())}
        x = torch.cat(list(x_dict_batched.values()))
        x = self.dense(x)
        output = self.softmax(x)
        # TODO:  try a layer that recive a dictonery(diffrent kind of nodes)_
        return output


class RamSimpleModel(pl.LightningModule):
    def __init__(self, model_arch: torch.nn.Module, **kwargs):
        super().__init__()
        self.model_arch = model_arch
        self.static_edge_index_dict = None
        self.is_computational_graph_saved = False

    def training_step(self, batch, batch_idx):
        x_dict, y_t, edge_index_dict, batch_dict = batch.x_dict, batch.y, batch.edge_index_dict, batch.batch_dict
        # edge_index_dict = {key:torch.tile(x,[y.size()[0],1,1]) for key,x in self.static_edge_index_dict.items()}
        pred = self.model_arch(x_dict, edge_index_dict, batch_dict)
        y_t_onehot = torch.tensor(torch.nn.functional.one_hot(y_t, 2), dtype=torch.float32)
        train_loss = torch.nn.functional.binary_cross_entropy(pred, y_t_onehot)
        correct = pred.argmax(dim=1).eq(y_t).sum().item()
        total = pred.size()[0]

        logs = {"train_loss": train_loss}
        batch_logs_dictionary = {
            "loss": train_loss,
            "log": logs,
            "correct": correct,
            "total": total,
        }
        return batch_logs_dictionary

    def training_epoch_end(self, outputs) -> None:
        avg_loss = torch.stack([xx['loss'] for xx in outputs]).mean()
        correct = sum([x["correct"] for x in outputs])
        total = sum([x["total"] for x in outputs])

        self.logger.experiment.add_scalar("Loss/Train", avg_loss, self.current_epoch)
        self.logger.experiment.add_scalar("Accuracy/Train", correct / total, self.current_epoch)
        self.save_computational_graph()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def lazy_initialize(self, sample):
        # x_dict = {key:torch.tile(x,[batch_size,1,1]) for key,x in sample.x_dict.items()}
        # edge_index_dict = {key:torch.tile(x,[batch_size,1,1]) for key,x in sample.edge_index_dict.items()}
        batch_dict = {key: torch.zeros(xx.size()[0], dtype=torch.long) for key, xx in sample.x_dict.items()}
        with torch.no_grad():  # Initialize lazy modules.
            _ = self.model_arch(sample.x_dict, sample.edge_index_dict, batch_dict=batch_dict)
        # self.logger.experiment.add_graph(RamSimpleModel(self.model_arch),(sample.x_dict, sample.edge_index_dict,batch_dict))


    def train(self, sample):
        self.lazy_initialize(sample)

    # def save_computational_graph(self,sample):
    #     batch_dict = {key: torch.zeros(xx.size()[0], dtype=torch.long) for key, xx in sample.x_dict.items()}
        # self.logger.experiment.add_graph(RamSimpleModel(self.model_arch),(sample.x_dict, sample.edge_index_dict,batch_dict))


    def set_static_edge_index(self, edge_index):
        self.static_edge_index_dict = edge_index

<IPython.core.display.Javascript object>

In [ ]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
synthetic_data_module = SyntheticDataModule()
model_arch = HeteroGNNArch(hidden_channels=24, out_channels=2, num_layers=2)
ram_model = RamSimpleModel(model_arch=model_arch)
ram_model.lazy_initialize(synthetic_data_module.get_single_sample())
ram_model.set_static_edge_index(synthetic_data_module.static_edge_index_dict)

logs_root = f"{str((Path.cwd().parent / 'logs').absolute())}"
tb_logger = pl.loggers.tensorboard.TensorBoardLogger(save_dir=logs_root,
                                                     name=datetime.now().strftime("%d-%m-%Y-%H-%M"))
trainer = pl.Trainer(logger=tb_logger)

trainer.fit(ram_model, synthetic_data_module)
a = 1

/Users/ramshirazi/opt/anaconda3/envs/ram_dev/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


AttributeError: 'NoneType' object has no attribute 'experiment'

<IPython.core.display.Javascript object>

In [ ]:
# export
from nbdev.export import notebook2script;

notebook2script()
#
# notebook2script()